In [0]:
import tensorflow as tf
import numpy as np
import keras

Using TensorFlow backend.


In [0]:
s=tf.Session()

Input tensor contains the input sequence and therefore determines the number of time steps
shape of x --> [**batch_size, input_sequence, embedding_size**]</br>
input_sequence = number of time steps

In [0]:
at_type='additive'
batch_size=32
e_input_sequence=10
d_input_sequence=6
d=50 #embedding size

In [0]:
x=tf.random_uniform(shape=[batch_size, e_input_sequence, d],minval=-1,maxval=1,dtype=tf.float32)
y=tf.random_uniform(shape=[batch_size, d_input_sequence, d],minval=-1,maxval=1,dtype=tf.float32)

##Understanding keras LSTM() 
lstm1, state_h, state_c = LSTM(1, return_state=True)
</br>**lstm1** ----> hidden state sequence of LSTM at each time-step. &nbsp;&nbsp;if **return_sequence= True** 
</br>**lstm1** ----> hidden state output of LSTM after all time-steps. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;if **return_sequence= False** 
</br>**state_h**---> hidden state output of LSTM after all time-steps
</br>**state_c** ----> cell state of the LSTM cell after the last time-step

In [0]:
e_out_dims=x.get_shape().as_list()[2]
d_out_dims=y.get_shape().as_list()[2]

out_dims specifies the dimensionality of the output of the LSTM, i.e dimensionality of the hidden state.

##Encoder

In [0]:
f_encoder = tf.keras.layers.CuDNNLSTM(e_out_dims,return_sequences=True,return_state=True)
b_encoder = tf.keras.layers.CuDNNLSTM(e_out_dims,return_sequences=True, return_state=True,go_backwards=True)

In [0]:
fw_encoder_seq,_, fw_encoder_state = f_encoder(x)
bw_encoder_seq,_,bw_encoder_state = b_encoder(x)
#return sequences ----> hidden state value at each timestep is returned. Used when stacking LSTMs
concat_encoder_seq=tf.concat([fw_encoder_seq,bw_encoder_seq],2)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
concat_encoder_seq.get_shape().as_list()

[32, 10, 100]

In [0]:
embed=tf.reshape(concat_encoder_seq,[-1,2*d])
squisher = tf.Variable(tf.random_uniform([2*d,d],minval=-1,maxval=1),dtype=tf.float32)
encoder_seq=tf.matmul(embed,squisher)
encoder_seq=tf.reshape(encoder_seq,[batch_size,e_input_sequence,d])
encoder_seq.get_shape().as_list()

[32, 10, 50]

In [0]:
encoder_seq[:,-1,:].get_shape().as_list()

[32, 50]

#Decoder

In [0]:
#####################################################just input one token here. Instead of y
decoder=tf.keras.layers.CuDNNLSTM(d_out_dims,return_state=True)


In [0]:
decoder_seq,decoder_state = decoder(y,initial_state=[encoder_seq[:,-1,:],fw_encoder_state])

ValueError: ignored

#Why attention?
##What's the shortcoming of seq2seq?
So far we've looked at encoder and decoder networks (both are LSTM networks here).
Seq2seq transforms input sequence (source) to a target output sequence.
In vanilla encoder-decoder models, only the last state of the encoder LSTM is used by the decoder.</br>
![alt text](https://lilianweng.github.io/lil-log/assets/images/encoder-decoder-example.png =500x)
</br>Consequently seq2seq models tend to forget the source words that came long before the final state. In other words, seq2seq models "forget" long source sentences.

##How attention mechanism addresses this shortcoming?
Qouted from [here](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)
>"In a nutshell, attention in the deep learning can be broadly interpreted as a vector of importance weights: in order to predict or infer one element, such as a pixel in an image or a word in a sentence, we estimate using the attention vector how strongly it is correlated with (or “attends to” as you may have read in many papers) other elements and take the sum of their values weighted by the attention vector as the approximation of the target."
>
![alt text](https://lilianweng.github.io/lil-log/assets/images/sentence-example-attention.png =500x)</br>
</br>Instead of only initialising the decoder with final state of the encoder, the context vector aka "thought" vector is the average of all encoder hidden states weighted by attention. </br>Note: the attention weights vary for the different decoder time steps.</br>Thus, the entire source input is used at each stage of the decoder wherein, different source words attend to different target words by varying degrees as specified by the attention (alignment) score matrix.</br>The matrix of alignment scores is a nice byproduct to explicitly show the correlation between source and target words.
</br>![alt text](https://lilianweng.github.io/lil-log/assets/images/bahdanau-fig3.png =500x)

In [0]:
sesh=tf.Session()
init=tf.global_variables_initializer()
sesh.run(init)
print(sesh.run(encoder_seq).shape)
print(sesh.run(decoder_seq).shape)

(32, 10, 50)
(32, 6, 50)


In [0]:
s=decoder_seq[:,1,:]
s=tf.expand_dims(s,1)
multiply=tf.constant([1,e_input_sequence,1])
s=tf.tile(s,multiply)
e=encoder_seq*s
e=tf.reduce_sum(e,axis=2)


### replace e with alpha in original code
e=tf.reshape(e,[-1,1])
temp=tf.reshape(encoder_seq,[-1,e_out_dims])
mul=tf.constant([1,e_out_dims])
e=tf.tile(e,mul)
weighted=e*temp
weighted=tf.reshape(weighted,[batch_size,e_input_sequence,e_out_dims])
weighted=tf.reduce_sum(weighted,axis=1)
weighted.get_shape().as_list()

[32, 50]

In [0]:
a=tf.constant([['b1_h1'],['b1_h2'],['b2_h1'],['b2_h2']])
b=tf.reshape(a,[2,2,1])
sesu=tf.Session()
sesu.run(a)

array([[b'b1_h1'],
       [b'b1_h2'],
       [b'b2_h1'],
       [b'b2_h2']], dtype=object)

In [0]:
sesu.run(b)[0]

array([[b'b1_h1'],
       [b'b1_h2']], dtype=object)

In [0]:
for i in range(batch_size):
  for t in range(d_input_sequence):
    H = encoder_seq[i].get_shape().as_list()
    s = decoder_seq[i,t] #################edit this after editing decoder network input
    ctx = get_context(H,s,at_type)
    s,c = tf.keras.layers.CuDNNLSTM(d_out_dims,return_state=True)(next_word,[ctx,decoder_state])


[10, 50]

In [0]:
dec

Defining the score function

In [0]:
#function that calulates score based on the type of attention
def score(encoder_states,s,att_type):
  f=32 #hyperparamter of sorts
  n=encoder_states.get_shape().as_list()[1]
  m=s.get_shape().as_list()
  d=encoder_states.get_shape().as_list()[0]
  if att_type=='additive':
    W=tf.Variable(tf.random_uniform(shape=[f,2*d],minval=-1,maxval=1))
    v=tf.Variable(tf.random_uniform(shape=[1,f],minval=-1,maxval=1))
    mat_s=tf.tile(s,[1,n])
    hs=tf.stack(encoder_states,mat_s)
    temp=tf.nn.tanh(tf.matmul(W,[]))
  if att_type=='multiplicative':
    #s will be [32,1,50]
    s=tf.expand_dims(s,1)
    multiply=tf.constant([1,e_input_sequence,1])
    alphas=tf.reduce_sum(encoder_seq*s,axis=2)
  if att_type=='sdot':
    s=tf.expand_dims(s,1)
    multiply=tf.constant([1,e_input_sequence,1])
    alphas=tf.reduce_sum(encoder_states*s,axis=2)
    alphas=1/(e_input_sequence**0.5)
    print("bitch")#code goes here
  if att_type=='kv':
    #code goes here
    print("fkjden")
    
  return alphas # return a [batch_size x n] vector
    #returns the softmax vector of dimensions = no. of encoder time steps

In [0]:
#function to construct context vector for given decoder state
def get_context(encoder_states,prev_decoder_state,attention):
  alpha=score(encoder_states,prev_decoder_state,attention)
  alpha=tf.reshape(alpha,[-1,1])
  temp=tf.reshape(encoder_states,[-1,e_out_dims])
  mul=tf.constant([1,e_out_dims])
  alpha=tf.tile(alpha,mul)
  ctx=alpha*temp
  ctx=tf.reshape(ctx,[batch_size,e_input_sequence,e_out_dims])
  ctx=tf.reduce_sum(ctx,axis=1)
  
  return ctx  #returns context vectors for all batches for this time-step
  #if n = number of source words
  #encoder_states = [batch_size,n,embedding_size]
  #alpha = [bactch_size,n]
  #ctx = [batch_size,embedding_size]
  

In [0]:
decoder_seq[:,1,:].get_shape().as_list()

[32, 50]

In [0]:
weighted.get_shape().as_list()

[32, 50]

In [0]:
s=tf.Session()
a = tf.constant(np.array([[.1, .3, .5, .9],[.1, .3, .5, .9]]))
t= tf.constant(np.array([[1, 0, 0, 0],[0, 0, 1, 0]]))  
loss_=tf.nn.softmax(a)
s.run(loss_)

array([[0.16838508, 0.205666  , 0.25120102, 0.37474789],
       [0.16838508, 0.205666  , 0.25120102, 0.37474789]])

In [0]:
#tester
np.log(0.16838508)

-1.7815017796738182

In [0]:
gloss=tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=t,logits=a),axis=0)
print(s.run(gloss))
#[[ 0.16838508  0.205666    0.25120102  0.37474789]]

3.163003538329237


In [0]:
s.run(t)
t.get_shape().as_list()

[2, 4]

In [0]:
t_vocab_size=30000
init=tf.global_variables_initializer()
s.run(init)
r=tf.concat([decoder_seq[:,1,:],weighted],axis=1)
d=r.get_shape().as_list()[1]
W_loss=tf.Variable(tf.random_uniform([d,t_vocab_size],minval=-1,maxval=1))
vec_loss = tf.matmul(r,W_loss)
vec_loss = tf.nn.softmax(vec_loss)
vec_loss.get_shape().as_list()

[32, 30000]

In [0]:
def loss(ctx,s,target_word):   #pass the target word as [batch_size, t_vocab_size] one hot vector
  r=tf.concat([s,ctx],axis=1)
  d=r.get_shape().as_list()[1]
  t_vocab_size=target_word.get_shape().as_list()[0]
  W_loss=tf.Variable(tf.random_uniform([d,t_vocab_size],minval=-1,maxval=1))
  vec_loss = tf.matmul(r,W_loss)
  loss_val = tf.nn.softmax_cross_entropy_with_logits(labels=target_word,logits=vec_loss)
   
  return loss_val

In [0]:
decoder_seq.get_shape().as_list()

[32, 6, 50]

In [0]:
a=tf.constant([[2],[3]])
b=tf.constant([[[2],[2]],[[2],[1]],[[5],[2]]])
mat_a=tf.tile(a,[1,10])
b.get_shape().as_list()

[3, 2, 1]

In [0]:
sess=tf.Session()
init_=tf.global_variables_initializer()
sess.run(init_)
sess.run(a)
sess.run(b)

array([[2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]], dtype=int32)

In [0]:
def test(a=10):
  print(a*b)

In [0]:
global b
b=10
test()
test(5)

100
50
